In [1]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from calendar import monthrange
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import numpy as np


In [2]:
# Masquer les SettingWithCopyWarning
pd.options.mode.chained_assignment = None

In [3]:
df = pd.read_csv("https://jedha-final-project-jrat.s3.amazonaws.com/datameteo_france_1950-2022_clean_04.csv")

In [4]:
df_stations_authorized = df[df["is_authorized"] == True]

In [5]:
df_stations_old = df_stations_authorized.groupby(["NUM_POSTE"]).max()
print(len(df_stations_old))

412


In [6]:
# filter out station qui n'ont pas de données récentes (2022)
last_year = df["Year"].max()
mask = df_stations_old["Year"]!=last_year
df_stations_old = df_stations_old[mask]
stations_old = df_stations_old.index.to_list()
df_recent = df_stations_authorized[~df_stations_authorized["NUM_POSTE"].isin(stations_old)]

In [7]:
print(len(df_recent.groupby(["NUM_POSTE"]).max()))

274


In [8]:
# la précédente exploration nous a permis de conclure que pour prédire à horizon 7 ans
# le modèle ne performait (MAPE moyenne de 13%) qu'à partir de 168 mois (14 ans) de continuité des données
# on filtre donc sur les stations qui ont 12 mois sur chaque année entre 2022 et 2009 inclus au minimum

stations = df_recent["NUM_POSTE"].unique()
to_drop = []

for num_station in stations:
    df_station = df_recent[df_recent["NUM_POSTE"]==num_station]

    for year in range(last_year,last_year-14,-1):
        nb_missing_month = 0
        try:
            df_station_year = df_station[df_station["Year"]==year]
            nb_missing_month = 12 - len(df_station_year["Month"].unique().tolist())
            if nb_missing_month > 0:
                to_drop.append(num_station)
                print(f"Drop station N°{num_station}")
                print(f"{nb_missing_month} manquants en {year}\n\n")
            else:
                print(f"Station N°{num_station} has {df_station_year["Month"].count()} months in {year}")

        except Exception as e:
            to_drop.append(num_station)
            print(e)
            print()
            print(f"Drop station N°{num_station}")
            print(f"{year} est manquante\n\n")

    print()

Station N°1014002 has 12 months in 2022
Station N°1014002 has 12 months in 2021
Station N°1014002 has 12 months in 2020
Station N°1014002 has 12 months in 2019
Station N°1014002 has 12 months in 2018
Station N°1014002 has 12 months in 2017
Station N°1014002 has 12 months in 2016
Station N°1014002 has 12 months in 2015
Station N°1014002 has 12 months in 2014
Station N°1014002 has 12 months in 2013
Station N°1014002 has 12 months in 2012
Station N°1014002 has 12 months in 2011
Station N°1014002 has 12 months in 2010
Station N°1014002 has 12 months in 2009

Station N°1071001 has 12 months in 2022
Station N°1071001 has 12 months in 2021
Station N°1071001 has 12 months in 2020
Station N°1071001 has 12 months in 2019
Station N°1071001 has 12 months in 2018
Station N°1071001 has 12 months in 2017
Station N°1071001 has 12 months in 2016
Station N°1071001 has 12 months in 2015
Station N°1071001 has 12 months in 2014
Station N°1071001 has 12 months in 2013
Station N°1071001 has 12 months in 2012

In [9]:
# gestion des outliers sur la target ?


In [10]:
df_recent = df_recent[~df_recent["NUM_POSTE"].isin(to_drop)]

In [11]:
# 648 stations qui ont au moins 8 années de continuité (12 mois complets) à partir de 2022
print(len(df_recent.groupby(["NUM_POSTE"]).max()))

203


In [12]:
# PREPROCESSING DATE

df_recent["AAAAMM"] = pd.to_datetime(df_recent["AAAAMM"])

def last_day_of_month(aaaamm):
    """
    prend une date format AAAAMM et retourne une string format AAAA-MM-DD avec DD = dernier jour du mois
    """
    year = aaaamm.year
    month = aaaamm.month
    last_day = monthrange(year, month)[1]
    return f"{year}-{month:02d}-{last_day:02d}"

df_recent["date"] = df_recent["AAAAMM"].apply(last_day_of_month)
df_recent["date"] = pd.to_datetime(df_recent["date"])


In [13]:
to_keep = ["NUM_POSTE","NOM_USUEL","LAT","LON","ALTI","Year","Month","vent_speed_inst_moy_mensu","departement_num","departement_name","region","date"]
to_drop = [col for col in df if col not in to_keep]

In [14]:
df_recent = df_recent.drop(to_drop,axis=1)

In [15]:
def clean_continuity(dataframe):

    """
    INPUT = un dataframe, le nom de sa variable date
    --> parcours la série d'année en année en commençant par la plus récente
    --> si année en cours < 12 mois, drop l'année et toutes les années antérieures
    --> si "trou" entre année en cours et année suivante, drop N-1 et toutes les années antérieures 
    --> transforme en série temporelle à frequence mensuelle
    
    OUTPUT = série temporelle clean
    """

    print(f"\nCleaning...")

    #--> check continuité des années : stop_year = N si N < 12 mois, N-1 si "trou" entre N et N-1
    years= df_station["Year"].unique().tolist()
    years.sort(reverse=True)

    month_unique = []
    stop_year = 0

    for index, year in enumerate(years):
        df_station_year = df_station[df_station["Year"] == year]
        month_unique = df_station_year["Month"].unique().tolist()
        if len(month_unique)!=12:
            stop_year = year
            print(f"Année : {year}. Continuité rompue.")
            print(f"Mois: {len(month_unique)} / 12.")
            break
        else:
            try:
                if year - years[index+1] != 1:
                    stop_year = years[index+1]
                    print(f"Année : {years[index+1]}. Continuité rompue.")
                    print(f"Année manquante : {years[index+1]}")
                    break
            except IndexError:
                print("Continuité garantie")
                pass

    #--> drop
    if stop_year > 0:
        dataframe = dataframe[dataframe["Year"] > stop_year]

        print(f"Données antérieures à {stop_year+1} supprimées.")
    else:
        print(f"Aucune rupture de continuité constatée, toutes les données ont été conservées.")
    

    #--> transforme en série temporelle à frequence mensuelle
    dataframe.set_index("date", inplace=True)
    dataframe = dataframe.asfreq("ME")

    #--> check NA
    check = "NOT OK" if dataframe.isna().any().any() else "OK"
    print(f"Check valeurs manquantes : {check}")
    print(f"...Terminé.")

    return dataframe
    

In [16]:
def get_wind_forecast(time, data):

    """
    INPUT =  nombre de mois (time) à prédire et le set de données

    --> prédit la force du vent à horizon "time"
    
    OUTPUT =  le df de test, les prédictions, la MAE et la MAPE
    """

    print(f"\nForecast à horizon {time/12} ans...")
    # calcule date de split 
    date_limite = data.index.max() - pd.DateOffset(months=time)

    # split train / test
    train = data[data.index <= date_limite]
    test = data[data.index > date_limite]

    # split variables exogènes / target
    to_keep = ["LON","LAT","ALTI"]

    var_exog_train = train[to_keep]
    y_train = train["vent_speed_inst_moy_mensu"]

    var_exog_test = test[to_keep]
    y_test = test["vent_speed_inst_moy_mensu"]

    # training
    model = SARIMAX(y_train, exog=var_exog_train, order=(1, 1, 2), seasonal_order=(2, 1, 2, 12), enforce_stationarity=False)
    sarima_model = model.fit(maxiter=1000)

    # prédictions
    forecast = sarima_model.get_forecast(steps = time, exog = var_exog_test)
    predicted_values = forecast.predicted_mean

    # évaluation
    mae = mean_absolute_error(y_test, predicted_values)
    mape = mean_absolute_percentage_error(y_test, predicted_values)

    # rendu
    print("...Terminé.")
    return test, predicted_values, mae, mape


In [17]:
# créé un dictionnaire contenant un dataframe par numéro de station
# clean continuité des dates sur le dataframe + transforme en série temporelle (clean_continuity)
# prédit (get_wind_forecast) et enregistre les prédictions horizon, la MAE et la MAPE générale
# enregistre le dataframe clean et le dataframe d'évaluation des prédictions

stations = df_recent["NUM_POSTE"].unique()
horizon = 84

dict_stations = {}

for station in stations:
    print(f"\n\n ======> Station : [{station}] <======")
    df_station = df_recent[df_recent["NUM_POSTE"]==station]
    df_station = clean_continuity(df_station)

    df_station_eval, predictions, mae, mape = get_wind_forecast(horizon,df_station)
    df_station_eval[f"pred_{horizon}_mois"] = predictions
    df_station_eval["mae"] = mae
    df_station_eval["mape"] = mape

    dict_stations[station] = [df_station,df_station_eval]






 ======> Station : [1014002] <======

Cleaning...
Année : 2004. Continuité rompue.
Mois: 10 / 12.
Données antérieures à 2005 supprimées.
Check valeurs manquantes : OK
...Terminé.

Forecast à horizon 7.0 ans...
...Terminé.


 ======> Station : [1071001] <======

Cleaning...
Année : 2002. Continuité rompue.
Mois: 8 / 12.
Données antérieures à 2003 supprimées.
Check valeurs manquantes : OK
...Terminé.

Forecast à horizon 7.0 ans...
...Terminé.


 ======> Station : [1414001] <======

Cleaning...
Année : 2002. Continuité rompue.
Mois: 4 / 12.
Données antérieures à 2003 supprimées.
Check valeurs manquantes : OK
...Terminé.

Forecast à horizon 7.0 ans...
...Terminé.


 ======> Station : [2037002] <======

Cleaning...
Continuité garantie
Aucune rupture de continuité constatée, toutes les données ont été conservées.
Check valeurs manquantes : OK
...Terminé.

Forecast à horizon 7.0 ans...
...Terminé.


 ======> Station : [2173002] <======

Cleaning...
Année : 1998. Continuité rompue.
Mois: 3 /

c:\Users\alexm\anaconda3\envs\jedha_project\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


...Terminé.


 ======> Station : [4209005] <======

Cleaning...
Année : 1998. Continuité rompue.
Mois: 6 / 12.
Données antérieures à 1999 supprimées.
Check valeurs manquantes : OK
...Terminé.

Forecast à horizon 7.0 ans...
...Terminé.


 ======> Station : [5046001] <======

Cleaning...
Année : 1957. Continuité rompue.
Mois: 11 / 12.
Données antérieures à 1958 supprimées.
Check valeurs manquantes : OK
...Terminé.

Forecast à horizon 7.0 ans...
...Terminé.


 ======> Station : [5055001] <======

Cleaning...
Année : 1997. Continuité rompue.
Mois: 6 / 12.
Données antérieures à 1998 supprimées.
Check valeurs manquantes : OK
...Terminé.

Forecast à horizon 7.0 ans...
...Terminé.


 ======> Station : [5061009] <======

Cleaning...
Année : 2003. Continuité rompue.
Mois: 1 / 12.
Données antérieures à 2004 supprimées.
Check valeurs manquantes : OK
...Terminé.

Forecast à horizon 7.0 ans...
...Terminé.


 ======> Station : [5070003] <======

Cleaning...
Année : 1992. Continuité rompue.
Mois: 1 / 

c:\Users\alexm\anaconda3\envs\jedha_project\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'


...Terminé.


 ======> Station : [6094002] <======

Cleaning...
Année : 2002. Continuité rompue.
Mois: 2 / 12.
Données antérieures à 2003 supprimées.
Check valeurs manquantes : OK
...Terminé.

Forecast à horizon 7.0 ans...
...Terminé.


 ======> Station : [8145001] <======

Cleaning...
Année : 2002. Continuité rompue.
Mois: 8 / 12.
Données antérieures à 2003 supprimées.
Check valeurs manquantes : OK
...Terminé.

Forecast à horizon 7.0 ans...
...Terminé.


 ======> Station : [8401001] <======

Cleaning...
Année : 2004. Continuité rompue.
Mois: 10 / 12.
Données antérieures à 2005 supprimées.
Check valeurs manquantes : OK
...Terminé.

Forecast à horizon 7.0 ans...
...Terminé.


 ======> Station : [9161003] <======

Cleaning...
Année : 2002. Continuité rompue.
Mois: 8 / 12.
Données antérieures à 2003 supprimées.
Check valeurs manquantes : OK
...Terminé.

Forecast à horizon 7.0 ans...
...Terminé.


 ======> Station : [10057001] <======

Cleaning...
Continuité garantie
Aucune rupture de cont

...

...Terminé.


 ======> Station : [89333001] <======

Cleaning...
Année : 2005. Continuité rompue.
Mois: 5 / 12.
Données antérieures à 2006 supprimées.
Check valeurs manquantes : OK
...Terminé.

Forecast à horizon 7.0 ans...


c:\Users\alexm\anaconda3\envs\jedha_project\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


...Terminé.


 ======> Station : [89365001] <======

Cleaning...
Année : 2005. Continuité rompue.
Mois: 8 / 12.
Données antérieures à 2006 supprimées.
Check valeurs manquantes : OK
...Terminé.

Forecast à horizon 7.0 ans...
...Terminé.


 ======> Station : [89380001] <======

Cleaning...
Année : 2003. Continuité rompue.
Mois: 7 / 12.
Données antérieures à 2004 supprimées.
Check valeurs manquantes : OK
...Terminé.

Forecast à horizon 7.0 ans...
...Terminé.


 ======> Station : [89387002] <======

Cleaning...
Année : 1986. Continuité rompue.
Mois: 3 / 12.
Données antérieures à 1987 supprimées.
Check valeurs manquantes : OK
...Terminé.

Forecast à horizon 7.0 ans...
...Terminé.


 ======> Station : [89418007] <======

Cleaning...
Année : 2004. Continuité rompue.
Mois: 4 / 12.
Données antérieures à 2005 supprimées.
Check valeurs manquantes : OK
...Terminé.

Forecast à horizon 7.0 ans...
...Terminé.


 ======> Station : [91184001] <======

Cleaning...
Année : 1998. Continuité rompue.
Mois: 

c:\Users\alexm\anaconda3\envs\jedha_project\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\alexm\anaconda3\envs\jedha_project\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'


...Terminé.


In [18]:
# 2ème station de la liste, dataframe initial
dict_stations[stations[1]][0]

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,Year,Month,vent_speed_inst_moy_mensu,departement_num,departement_name,region
date,,,,,,,,,,,
2003-01-31,1071001,CESSY,46.310333,6.080333,507,2003,1,7.9,1,Ain,AUVERGNE RHONE ALPES
2003-02-28,1071001,CESSY,46.310333,6.080333,507,2003,2,6.8,1,Ain,AUVERGNE RHONE ALPES
2003-03-31,1071001,CESSY,46.310333,6.080333,507,2003,3,8.3,1,Ain,AUVERGNE RHONE ALPES
2003-04-30,1071001,CESSY,46.310333,6.080333,507,2003,4,10.1,1,Ain,AUVERGNE RHONE ALPES
2003-05-31,1071001,CESSY,46.310333,6.080333,507,2003,5,7.6,1,Ain,AUVERGNE RHONE ALPES
...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,1071001,CESSY,46.310333,6.080333,507,2022,8,7.9,1,Ain,AUVERGNE RHONE ALPES
2022-09-30,1071001,CESSY,46.310333,6.080333,507,2022,9,6.8,1,Ain,AUVERGNE RHONE ALPES
2022-10-31,1071001,CESSY,46.310333,6.080333,507,2022,10,5.4,1,Ain,AUVERGNE RHONE ALPES


In [19]:
# 2ème station de la liste, dataframe d'évaluation des prédictions
dict_stations[stations[1]][1]

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,Year,Month,vent_speed_inst_moy_mensu,departement_num,departement_name,region,pred_84_mois,mae,mape
date,,,,,,,,,,,,,,
2016-01-31,1071001,CESSY,46.310333,6.080333,507,2016,1,6.5,1,Ain,AUVERGNE RHONE ALPES,6.643471,1.074443,0.136078
2016-02-29,1071001,CESSY,46.310333,6.080333,507,2016,2,8.6,1,Ain,AUVERGNE RHONE ALPES,7.050499,1.074443,0.136078
2016-03-31,1071001,CESSY,46.310333,6.080333,507,2016,3,9.7,1,Ain,AUVERGNE RHONE ALPES,8.237794,1.074443,0.136078
2016-04-30,1071001,CESSY,46.310333,6.080333,507,2016,4,7.2,1,Ain,AUVERGNE RHONE ALPES,7.887070,1.074443,0.136078
2016-05-31,1071001,CESSY,46.310333,6.080333,507,2016,5,7.6,1,Ain,AUVERGNE RHONE ALPES,7.418024,1.074443,0.136078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,1071001,CESSY,46.310333,6.080333,507,2022,8,7.9,1,Ain,AUVERGNE RHONE ALPES,6.151875,1.074443,0.136078
2022-09-30,1071001,CESSY,46.310333,6.080333,507,2022,9,6.8,1,Ain,AUVERGNE RHONE ALPES,6.184310,1.074443,0.136078
2022-10-31,1071001,CESSY,46.310333,6.080333,507,2022,10,5.4,1,Ain,AUVERGNE RHONE ALPES,5.375272,1.074443,0.136078


In [20]:
# construction du dataframe de comparaison inter station

num_stations = []
nb_month_available = []
lat = []
lon = []
alti = []
true_val = []
pred_val = []
mae = []
mape = []

for key, value in dict_stations.items():
    num_stations.append(key)
    nb_month_available.append(len(value[0]))
    date_pred = value[1].index.max()

    data = value[1][value[1].index==date_pred]

    lat.append(data.loc[date_pred,"LAT"])
    lon.append(data.loc[date_pred,"LON"])
    alti.append(data.loc[date_pred,"ALTI"])
    true_val.append(data.loc[date_pred,"vent_speed_inst_moy_mensu"])
    pred_val.append(data.loc[date_pred,f"pred_{horizon}_mois"])
    mae.append(data.loc[date_pred,"mae"])
    mape.append(data.loc[date_pred,"mape"])

data = {
    "Num_station": num_stations,
    "Nb_mois_dispo": nb_month_available,
    "Latitude": lat,
    "Longitude": lon,
    "Altitude": alti,
    "Reel_2022-12": true_val,
    "Pred_2022-12": pred_val,
    "MAE": mae,
    "MAPE": mape,
}

df_eval_global = pd.DataFrame(data)


In [21]:
df_eval_global

,Num_station,Nb_mois_dispo,Latitude,Longitude,Altitude,Reel_2022-12,Pred_2022-12,MAE,MAPE
0,1014002,216,46.278167,5.669000,534,7.2,5.171361,0.941105,0.157790
1,1071001,240,46.310333,6.080333,507,5.8,6.000233,1.074443,0.136078
2,1414001,240,45.916167,5.624667,878,7.9,7.318747,1.102503,0.118098
3,2037002,408,49.595667,3.610333,78,14.0,15.796287,1.740910,0.146857
4,2173002,288,49.633833,3.197833,83,12.6,12.963039,1.214182,0.104954
...,...,...,...,...,...,...,...,...,...
198,89380001,228,48.086500,3.104000,184,13.0,13.106171,1.418169,0.121515
199,89387002,432,48.167667,3.288667,70,10.4,11.620964,1.374829,0.159942
200,89418007,216,47.868167,3.995000,200,11.9,10.496367,1.487920,0.131908
201,91184001,288,48.421833,2.374333,71,11.2,9.981205,1.404430,0.177406


In [22]:
MAPE_by_month = df_eval_global.groupby(["Nb_mois_dispo"]).mean()
MAPE_by_month


,Num_station,Latitude,Longitude,Altitude,Reel_2022-12,Pred_2022-12,MAE,MAPE
Nb_mois_dispo,,,,,,,,
168,4.687400e+07,45.739834,5.067222,656.333333,7.100000,6.760067,1.016239,0.132501
180,7.006450e+07,50.150083,3.082416,78.500000,16.050000,18.763901,1.872811,0.135176
192,4.040314e+07,47.176071,4.441619,574.285714,10.728571,9.619828,1.279801,0.133198
204,5.507500e+07,47.167204,4.009093,201.222222,11.400000,11.400914,1.615551,0.141846
216,4.439611e+07,47.102009,2.929237,231.263158,11.378947,11.253283,1.291355,0.125079
228,4.928015e+07,46.943548,4.127059,369.785714,11.421429,11.608388,1.391388,0.132543
240,3.643483e+07,45.424833,3.361406,452.652174,9.752174,10.255388,1.287735,0.122054
252,5.130296e+07,46.948051,3.052487,325.307692,10.892308,10.201986,1.329888,0.135909
264,3.907947e+07,47.695500,4.309222,1005.000000,14.433333,14.614093,1.701769,0.120237


In [23]:
px.line(MAPE_by_month, x=MAPE_by_month.index, y="MAPE")

In [29]:
df_stat = dict_stations[stations[0]][1].reset_index()

In [34]:
df_stat_tot = pd.DataFrame(columns=df_stat.columns)
for key,value in dict_stations.items():
    df_stat = value[1].reset_index()
    df_stat_tot = pd.concat([df_stat_tot,df_stat], ignore_index=True)

C:\Users\alexm\AppData\Local\Temp\ipykernel_13980\843924546.py:4: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [35]:
df_stat_tot

,date,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,Year,Month,vent_speed_inst_moy_mensu,departement_num,departement_name,region,pred_84_mois,mae,mape
0,2016-01-31,1014002,ARBENT,46.278167,5.669,534,2016,1,6.1,1,Ain,AUVERGNE RHONE ALPES,4.282549,0.941105,0.157790
1,2016-02-29,1014002,ARBENT,46.278167,5.669,534,2016,2,7.2,1,Ain,AUVERGNE RHONE ALPES,4.768090,0.941105,0.157790
2,2016-03-31,1014002,ARBENT,46.278167,5.669,534,2016,3,6.5,1,Ain,AUVERGNE RHONE ALPES,6.228707,0.941105,0.157790
3,2016-04-30,1014002,ARBENT,46.278167,5.669,534,2016,4,5.8,1,Ain,AUVERGNE RHONE ALPES,6.461510,0.941105,0.157790
4,2016-05-31,1014002,ARBENT,46.278167,5.669,534,2016,5,6.8,1,Ain,AUVERGNE RHONE ALPES,6.085793,0.941105,0.157790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17047,2022-08-31,91200002,DOURDAN,48.526167,1.993,116,2022,8,6.8,91,Essonne,ILE DE FRANCE,5.436382,1.024165,0.129682
17048,2022-09-30,91200002,DOURDAN,48.526167,1.993,116,2022,9,6.1,91,Essonne,ILE DE FRANCE,5.962940,1.024165,0.129682
17049,2022-10-31,91200002,DOURDAN,48.526167,1.993,116,2022,10,5.8,91,Essonne,ILE DE FRANCE,5.488471,1.024165,0.129682
17050,2022-11-30,91200002,DOURDAN,48.526167,1.993,116,2022,11,7.2,91,Essonne,ILE DE FRANCE,7.188313,1.024165,0.129682


In [36]:
df_stat_tot.to_csv("true_pred_2016-2022.csv", index=False)

In [28]:
df_tot = pd.concat([df_stat,df_stat_2], ignore_index=True)
df_tot

,date,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,Year,Month,vent_speed_inst_moy_mensu,departement_num,departement_name,region,pred_84_mois,mae,mape
0,2016-01-31,1071001,CESSY,46.310333,6.080333,507,2016,1,6.5,1,Ain,AUVERGNE RHONE ALPES,6.643471,1.074443,0.136078
1,2016-02-29,1071001,CESSY,46.310333,6.080333,507,2016,2,8.6,1,Ain,AUVERGNE RHONE ALPES,7.050499,1.074443,0.136078
2,2016-03-31,1071001,CESSY,46.310333,6.080333,507,2016,3,9.7,1,Ain,AUVERGNE RHONE ALPES,8.237794,1.074443,0.136078
3,2016-04-30,1071001,CESSY,46.310333,6.080333,507,2016,4,7.2,1,Ain,AUVERGNE RHONE ALPES,7.887070,1.074443,0.136078
4,2016-05-31,1071001,CESSY,46.310333,6.080333,507,2016,5,7.6,1,Ain,AUVERGNE RHONE ALPES,7.418024,1.074443,0.136078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,2022-08-31,1414001,SUTRIEU,45.916167,5.624667,878,2022,8,9.7,1,Ain,AUVERGNE RHONE ALPES,8.233784,1.102503,0.118098
164,2022-09-30,1414001,SUTRIEU,45.916167,5.624667,878,2022,9,9.0,1,Ain,AUVERGNE RHONE ALPES,7.487826,1.102503,0.118098
165,2022-10-31,1414001,SUTRIEU,45.916167,5.624667,878,2022,10,7.2,1,Ain,AUVERGNE RHONE ALPES,7.113751,1.102503,0.118098
166,2022-11-30,1414001,SUTRIEU,45.916167,5.624667,878,2022,11,7.9,1,Ain,AUVERGNE RHONE ALPES,7.373382,1.102503,0.118098
